In [14]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [15]:
db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [16]:
# defining a function for executing the queries
def execute(table):
    return pd.io.sql.read_sql(table, con = engine)

In [17]:
query = """SELECT * 
                    FROM books 
                    ORDER BY num_pages desc"""
df_books=execute(query)
df_books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,298,236,Harry Potter Boxed Set Books 1-5 (Harry Potte...,2690,2004-09-13,253
1,684,354,The Complete Anne of Green Gables Boxed Set (A...,2088,1998-10-06,285
2,361,240,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,1728,2012-09-25,33
3,760,224,The Iliad/The Odyssey,1556,1999-11-01,217
4,685,57,The Complete Calvin and Hobbes,1456,2005-09-06,18


In [18]:
query ="""SELECT * 
                    FROM authors 
                    ORDER BY author_id desc"""
df_authors=execute(query)
df_authors.head()

,author_id,author
0,636,Åsne Seierstad/Ingrid Christopherson
1,635,Zora Neale Hurston
2,634,Zilpha Keatley Snyder
3,633,Zadie Smith
4,632,William Strunk Jr./E.B. White


In [19]:
query ="""SELECT * 
                    FROM publishers 
                    ORDER BY publisher_id desc"""
df_publishers=execute(query)
df_publishers.head()

,publisher_id,publisher
0,340,Yearling Books
1,339,Yearling
2,338,Yale University Press
3,337,Wyatt Book
4,336,Workman Publishing Company


In [20]:
query ="""SELECT * 
                    FROM reviews 
                    ORDER BY review_id desc"""
df_reviews=execute(query)
df_reviews.head()

,review_id,book_id,username,text
0,2793,1000,jarvispaul,Republican staff bit eat material measure plan...
1,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...
2,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
3,2790,1000,wknight,Change lose answer close pressure. Spend so now.
4,2789,999,martinadam,Later hospital turn easy community. Fact same ...


In [21]:
query ="""SELECT * 
                    FROM ratings 
                    ORDER BY rating_id desc"""
df_ratings=execute(query)
df_ratings.head()

,rating_id,book_id,username,rating
0,6456,1000,fharris,5
1,6455,1000,zross,2
2,6454,1000,jarvispaul,5
3,6453,1000,wendy18,4
4,6452,1000,carolrodriguez,4


In [22]:
# to find the number of books released after January 1, 2000
query1 = "SELECT count(*) as pub_count FROM books WHERE publication_date > '2000-01-01';"
result1 = execute(query1)
print(result1)

   pub_count
0        819


A total of 819 books where published after January 1, 2000

In [23]:
# to find the number of user reviews and the average rating for each book.
query2 = "SELECT reviews.book_id, count(distinct(review_id)) as review_count, avg(rating) as avg_rating FROM reviews JOIN ratings ON ratings.book_id = reviews.book_id GROUP BY 1"
result2 = execute(query2)
print(result2)

     book_id  review_count  avg_rating
0          1             2    3.666667
1          2             1    2.500000
2          3             3    4.666667
3          4             2    4.500000
4          5             4    4.000000
..       ...           ...         ...
989      996             3    3.666667
990      997             3    3.400000
991      998             4    3.200000
992      999             2    4.500000
993     1000             4    3.833333

[994 rows x 3 columns]


* There are 994 books with each having its own unique identification under 'book_id'. 
* 'review_count' represents the amount of times each book has been reviewed (from counting there unique review id)
* 'avg_rating' is the summing of all the ratings recieved by each book divided by the number of times it was rated.

In [24]:
# to Identify the publisher that has released the greatest number of books with more than 50 pages 
query3 = "SELECT publishers.publisher_id, publishers.publisher, count(books.publisher_id) as num_count FROM publishers JOIN books ON books.publisher_id = publishers.publisher_id WHERE num_pages > 50 GROUP BY 1,2 ORDER BY 3 DESC LIMIT 1"
result3 = execute(query3)
print(result3)

   publisher_id      publisher  num_count
0           212  Penguin Books         42


Every publisher has its unique id(publisher_id), penguin Books with id 212 is the publisher that has released the greatest number of books (42 books) with more that 50 pages.

In [25]:
# to identify the author with the highest average book rating (looking only at books with at least 50 ratings)
query4= """SELECT books.title, authors.author, books.author_id, avg(ratings.rating) as avg_rating FROM authors JOIN books ON books.author_id = authors.author_id JOIN ratings ON ratings.book_id = books.book_id GROUP BY 1,2,3 HAVING count(ratings.rating) >= 50 ORDER BY 4 DESC LIMIT 1"""
result4 = execute(query4)
print(result4)

                                               title  \
0  Harry Potter and the Prisoner of Azkaban (Harr...   

                       author  author_id  avg_rating  
0  J.K. Rowling/Mary GrandPré        236    4.414634  


J.K. Rowling/Mary GrandPré, with author id '236', recieved the highest average rating (avg_rating), having a total rating of 50 and above.

In [27]:
### Find the average number of text reviews among users who rated more than 50 books.
query5 = "SELECT AVG(review_count) AS avg_review_count FROM (SELECT username, COUNT(text) review_count FROM reviews GROUP BY username) AS t1 WHERE username IN (SELECT username FROM ratings GROUP BY username HAVING COUNT(rating_id) > 50)"
result5 = execute(query5)
print(result5)

   avg_review_count
0         24.333333


156 users have rated more than 50 books and their average number of text reviews is shown in the column "avg_reviews".